In [1]:
import torch
torch.manual_seed(1)

In [2]:
def synthetic_data(true_w, true_b, num_samples):
    X = torch.normal(0.0, 1.0, (num_samples, true_w.size(0)))
    y = torch.matmul(X, true_w) + true_b
    y += torch.normal(0.0, 0.01, y.size())
    y = torch.reshape(y, (-1, 1))
    return X,y

In [3]:
true_w = torch.tensor([2.0, -3.4], dtype=torch.float)
print(true_w)
print(true_w.size())
print(true_w.size(0))
true_b = 4.2
num_samples = 20

tensor([ 2.0000, -3.4000])
torch.Size([2])
2


In [4]:
X_ts, y_ts= synthetic_data(true_w, true_b, num_samples)

X_np = X_ts.numpy()
y_np = y_ts.numpy()

print(X_ts)
print(X_ts.shape)

print(y_ts)
print(y_ts.shape)

tensor([[-1.5256, -0.7502],
        [-0.6540, -1.6095],
        [-0.1002, -0.6092],
        [-0.9798, -1.6091],
        [-0.7121,  0.3037],
        [-0.7773, -0.2515],
        [-0.2223,  1.6871],
        [ 0.2284,  0.4676],
        [-0.6970, -1.1608],
        [ 0.6995,  0.1991],
        [ 0.8657,  0.2444],
        [-0.6629,  0.8073],
        [-1.8821, -0.7765],
        [ 2.0242, -0.0865],
        [ 0.0981, -1.2150],
        [ 0.7312,  1.1718],
        [ 2.4070,  0.2786],
        [ 0.2468,  1.1843],
        [-0.7282,  1.1633],
        [-0.0091, -0.8425]])
torch.Size([20, 2])
tensor([[ 3.7017],
        [ 8.3590],
        [ 6.0529],
        [ 7.6983],
        [ 1.7472],
        [ 3.5044],
        [-1.9781],
        [ 3.0778],
        [ 6.7476],
        [ 4.9232],
        [ 5.0939],
        [ 0.1366],
        [ 3.0614],
        [ 8.5587],
        [ 8.5282],
        [ 1.6656],
        [ 8.0694],
        [ 0.6618],
        [-1.2220],
        [ 7.0535]])
torch.Size([20, 1])


## 1.1 method1: statsmodels linear model

In [5]:
import statsmodels.api as sm

/home/sz/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/sz/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [6]:
X_with_one_np = sm.add_constant(X_np)

In [7]:
X_with_one_np

array([[ 1.        , -1.5255959 , -0.7502318 ],
       [ 1.        , -0.65398091, -1.60948479],
       [ 1.        , -0.10016718, -0.60918891],
       [ 1.        , -0.97977227, -1.60909629],
       [ 1.        , -0.71214461,  0.30372199],
       [ 1.        , -0.77731431, -0.25145525],
       [ 1.        , -0.22227049,  1.6871134 ],
       [ 1.        ,  0.22842517,  0.46763551],
       [ 1.        , -0.69697243, -1.16076148],
       [ 1.        ,  0.6995424 ,  0.19908163],
       [ 1.        ,  0.86569238,  0.2444039 ],
       [ 1.        , -0.66291136,  0.80730826],
       [ 1.        , -1.88214254, -0.77654505],
       [ 1.        ,  2.02420211, -0.0865412 ],
       [ 1.        ,  0.09812354, -1.21500671],
       [ 1.        ,  0.73116672,  1.17180479],
       [ 1.        ,  2.40697813,  0.27856624],
       [ 1.        ,  0.24675292,  1.18432665],
       [ 1.        , -0.72821373,  1.16327441],
       [ 1.        , -0.00907459, -0.84246337]])

In [8]:
result = sm.OLS(y_np, X_with_one_np).fit()

In [9]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.312e+06
Date:                Wed, 02 Nov 2022   Prob (F-statistic):           7.91e-45
Time:                        18:32:01   Log-Likelihood:                 67.394
No. Observations:                  20   AIC:                            -128.8
Df Residuals:                      17   BIC:                            -125.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1987      0.002   2070.536      0.0

In [10]:
result.params

array([ 4.19869842,  2.0028701 , -3.39948582])

In [11]:
y_predict = result.predict()
print(y_predict)
print(y_predict.shape)

[ 3.69353037  8.36028033  6.06900565  7.70644184  1.73986665  3.49665738
 -1.98179859  3.06648408  6.74874535  4.9230158   5.10172021  0.12654012
  3.06886527  8.54710787  8.52562519  1.67959663  8.07258096  0.66681081
 -1.21435393  7.04446549]
(20,)


In [12]:
print(y_np)

[[ 3.701748  ]
 [ 8.359044  ]
 [ 6.052874  ]
 [ 7.6983    ]
 [ 1.7471555 ]
 [ 3.5044036 ]
 [-1.9781481 ]
 [ 3.0778396 ]
 [ 6.747579  ]
 [ 4.9232044 ]
 [ 5.0938716 ]
 [ 0.13664566]
 [ 3.0614007 ]
 [ 8.558734  ]
 [ 8.528207  ]
 [ 1.6655995 ]
 [ 8.069377  ]
 [ 0.6617757 ]
 [-1.2219727 ]
 [ 7.0535493 ]]


In [13]:
#residual = y_true - y_predict
residuals = result.resid
print(residuals)
print(residuals.shape)

[ 0.00821752 -0.00123625 -0.01613156 -0.00814196  0.00728889  0.00774621
  0.00365049  0.01135553 -0.00116625  0.00018862 -0.00784861  0.01010554
 -0.00746462  0.01162607  0.00258164 -0.01399716 -0.00320401 -0.00503511
 -0.00761877  0.0090838 ]
(20,)


In [28]:
theoritical_residuals_np = y_np.squeeze(-1) - result.predict()
print(theoritical_residuals_np)

[ 0.00821752 -0.00123625 -0.01613156 -0.00814196  0.00728889  0.00774621
  0.00365049  0.01135553 -0.00116625  0.00018862 -0.00784861  0.01010554
 -0.00746462  0.01162607  0.00258164 -0.01399716 -0.00320401 -0.00503511
 -0.00761877  0.0090838 ]


## 1.2 method2: sklearn linear model

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [16]:
lin_reg = LinearRegression(fit_intercept=True)
lin_reg.fit(X_np, y_np)
print(lin_reg.intercept_, lin_reg.coef_)

[4.1986985] [[ 2.0028694 -3.399485 ]]


## 1.3 method3: use np.linalg.lstsq to get regression coefficients

In [17]:
numpy_output_tuple = np.linalg.lstsq(X_with_one_np, y_np, rcond=None)
print(numpy_output_tuple[0])

[[ 4.19869842]
 [ 2.0028701 ]
 [-3.39948582]]


## 1.4 method4: use torch.linalg.lstsq to get regression coefficients

In [18]:
X_with_one_ts = torch.cat([torch.ones((len(X_ts),1)), X_ts], 1)
print(X_with_one_ts)
print(X_with_one_ts.shape)

tensor([[ 1.0000, -1.5256, -0.7502],
        [ 1.0000, -0.6540, -1.6095],
        [ 1.0000, -0.1002, -0.6092],
        [ 1.0000, -0.9798, -1.6091],
        [ 1.0000, -0.7121,  0.3037],
        [ 1.0000, -0.7773, -0.2515],
        [ 1.0000, -0.2223,  1.6871],
        [ 1.0000,  0.2284,  0.4676],
        [ 1.0000, -0.6970, -1.1608],
        [ 1.0000,  0.6995,  0.1991],
        [ 1.0000,  0.8657,  0.2444],
        [ 1.0000, -0.6629,  0.8073],
        [ 1.0000, -1.8821, -0.7765],
        [ 1.0000,  2.0242, -0.0865],
        [ 1.0000,  0.0981, -1.2150],
        [ 1.0000,  0.7312,  1.1718],
        [ 1.0000,  2.4070,  0.2786],
        [ 1.0000,  0.2468,  1.1843],
        [ 1.0000, -0.7282,  1.1633],
        [ 1.0000, -0.0091, -0.8425]])
torch.Size([20, 3])


In [19]:
torch_output_named_tuple = torch.linalg.lstsq(X_with_one_ts, y_ts,-1)

In [23]:
intercepts_and_coefficient = torch_output_named_tuple.solution
print(intercepts_and_coefficient)
print(intercepts_and_coefficient.shape)

tensor([[ 4.1987],
        [ 2.0029],
        [-3.3995]])
torch.Size([3, 1])


In [22]:
squared_residuals = torch_output_named_tuple.residuals
print(squared_residuals)

tensor([])


In [25]:
y_pred_ts = X_with_one_ts.matmul(intercepts_and_coefficient)
print(y_pred_ts)

tensor([[ 3.6935],
        [ 8.3603],
        [ 6.0690],
        [ 7.7064],
        [ 1.7399],
        [ 3.4967],
        [-1.9818],
        [ 3.0665],
        [ 6.7487],
        [ 4.9230],
        [ 5.1017],
        [ 0.1265],
        [ 3.0689],
        [ 8.5471],
        [ 8.5256],
        [ 1.6796],
        [ 8.0726],
        [ 0.6668],
        [-1.2144],
        [ 7.0445]])


In [29]:
residuals_ts = y_ts - y_pred_ts
print(residuals_ts)
print("--------------------")
print(theoritical_residuals_np)

tensor([[ 0.0082],
        [-0.0012],
        [-0.0161],
        [-0.0081],
        [ 0.0073],
        [ 0.0077],
        [ 0.0037],
        [ 0.0114],
        [-0.0012],
        [ 0.0002],
        [-0.0078],
        [ 0.0101],
        [-0.0075],
        [ 0.0116],
        [ 0.0026],
        [-0.0140],
        [-0.0032],
        [-0.0050],
        [-0.0076],
        [ 0.0091]])
--------------------
[ 0.00821752 -0.00123625 -0.01613156 -0.00814196  0.00728889  0.00774621
  0.00365049  0.01135553 -0.00116625  0.00018862 -0.00784861  0.01010554
 -0.00746462  0.01162607  0.00258164 -0.01399716 -0.00320401 -0.00503511
 -0.00761877  0.0090838 ]
